# __ÁRVORES DE DECISÃO__

***
## _Prompts e Respostas_

` Prompt 1: Necessito que crie um algoritmo sem o uso de bibliotecas como o 'scikit-learn' para o método de aprendizado supervisionado Árvores de Decisão que lide com valores numéricos e categóricos na linguagem Python (Jupyter Notebook)`

>Claro, vou te orientar sobre como criar um algoritmo para Árvores de Decisão em Python sem usar bibliotecas como scikit-learn. Vamos começar definindo a estrutura básica de uma árvore de decisão e, em seguida, discutiremos como construir o algoritmo passo a passo.

>Uma árvore de decisão é uma estrutura hierárquica composta por nós de decisão e nós de folha. Em cada nó de decisão, uma pergunta é feita sobre um atributo do conjunto de dados e, dependendo da resposta, o fluxo segue por diferentes ramos da árvore. Nos nós de folha, uma decisão é tomada.

>Vamos criar um algoritmo simples para Árvores de Decisão:

>Divisão de Dados: Começamos dividindo nossos dados em conjuntos de treinamento e teste.
>Definição da Estrutura da Árvore: Definimos a estrutura da árvore, criando classes para nós de decisão e nós de folha.
>Cálculo da Impureza: Implementamos funções para calcular a impureza dos dados, como o índice de Gini ou a entropia.
>Criação da Árvore: Implementamos a lógica para construir a árvore de decisão recursivamente, dividindo os dados com base nos atributos e valores que maximizam a pureza.
>Predição: Implementamos a função para prever os rótulos das amostras de teste usando a árvore treinada.
>Vou começar criando a estrutura básica do código. Depois, podemos continuar com cada passo. Vamos lá:

In [2]:
import numpy as np

class Node:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index  # Índice do atributo usado para a divisão
        self.threshold = threshold  # Valor de corte para a divisão
        self.left = left  # Subárvore esquerda
        self.right = right  # Subárvore direita
        self.value = value  # Valor da classe se o nó é uma folha

class DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth

    def fit(self, X, y):
        self.n_classes = len(np.unique(y))
        self.n_features = X.shape[1]
        self.tree = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))

        # Critérios de parada para a recursão
        if (self.max_depth is not None and depth >= self.max_depth) or n_labels == 1:
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        # Encontrar a melhor divisão
        best_feat, best_thresh = self._best_criteria(X, y, n_samples, n_features)

        # Dividir os dados e crescer recursivamente
        left_indices, right_indices = self._split(X[:, best_feat], best_thresh)
        left = self._grow_tree(X[left_indices, :], y[left_indices], depth + 1)
        right = self._grow_tree(X[right_indices, :], y[right_indices], depth + 1)
        return Node(feature_index=best_feat, threshold=best_thresh, left=left, right=right)

    def _best_criteria(self, X, y, n_samples, n_features):
        best_gini = np.inf
        best_feat = None
        best_thresh = None

        for feat_index in range(n_features):
            thresholds = np.unique(X[:, feat_index])
            for threshold in thresholds:
                left_indices, right_indices = self._split(X[:, feat_index], threshold)
                if len(left_indices) == 0 or len(right_indices) == 0:
                    continue
                gini = self._gini_impurity(y[left_indices], y[right_indices])
                if gini < best_gini:
                    best_gini = gini
                    best_feat = feat_index
                    best_thresh = threshold
        return best_feat, best_thresh

    def _gini_impurity(self, left_labels, right_labels):
        p_left = len(left_labels) / (len(left_labels) + len(right_labels))
        p_right = len(right_labels) / (len(left_labels) + len(right_labels))
        gini = 1.0 - (p_left**2 + p_right**2)
        return gini

    def _split(self, feature, threshold):
        left_indices = np.where(feature <= threshold)[0]
        right_indices = np.where(feature > threshold)[0]
        return left_indices, right_indices

    def _most_common_label(self, y):
        return np.bincount(y).argmax()

    def predict(self, X):
        return [self._traverse_tree(x, self.tree) for x in X]

    def _traverse_tree(self, x, node):
        if node.value is not None:
            return node.value
        feature_value = x[node.feature_index]
        if feature_value <= node.threshold:
            return self._traverse_tree(x, node.left)
        else:
            return self._traverse_tree(x, node.right)

>Esse é um começo para o algoritmo de Árvores de Decisão. Precisamos ainda testar e validar o código, além de implementar funções para calcular métricas de desempenho, como precisão, recall e F1-score. Vamos continuar a partir daqui!

`Prompt 2: Como processo  os dados do dataset "penguins" para utilizar neste algoritmo sem utilizar bibliotecas como sklearn? `
>Para processar os dados do conjunto de dados penguins sem utilizar bibliotecas como Scikit-learn, você precisará carregar os dados e convertê-los para um formato compatível com o seu algoritmo de árvore de decisão. Aqui está um exemplo de como você pode fazer isso:

In [3]:
import pandas as pd
import numpy as np

# Carregar os dados dos pinguins
penguins_data = pd.read_csv('penguins.csv')

# Pré-processamento dos dados
# Remover linhas com valores ausentes
penguins_data.dropna(inplace=True)

# Mapear as espécies para valores numéricos
species_mapping = {'Adelie': 0, 'Chinstrap': 1, 'Gentoo': 2}
penguins_data['species'] = penguins_data['species'].map(species_mapping)

# Selecionar as características (atributos) e as classes (rótulos)
X = penguins_data[['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']].values
y = penguins_data['species'].values

# Agora você pode usar X e y como entrada para o seu algoritmo de árvore de decisão
# Certifique-se de que o algoritmo aceita arrays NumPy como entrada.
# Por exemplo, você pode usar algo assim:
decision_tree = DecisionTree(max_depth=3)
decision_tree.fit(X, y)
redictions = decision_tree.predict(X)


>Este código carrega os dados do arquivo CSV "penguins.csv", remove as linhas com valores ausentes, mapeia as espécies para valores numéricos, seleciona as características e as classes e as converte em arrays NumPy para uso com o seu algoritmo de árvore de decisão. Certifique-se de substituir 'penguins.csv' pelo caminho do seu arquivo de dados.

` Prompt 3: como posso utilizar o dataset Penguins, sem o uso de bibliotecas como 'scikit-learn' para testar e treinar esse conjunto?`

>Você pode treinar e testar um modelo usando o conjunto de dados "Penguins" sem o uso de bibliotecas como scikit-learn escrevendo seu próprio código para isso. Aqui está uma abordagem simples usando uma divisão de treinamento/teste e um modelo de árvore de decisão que você já possui:

In [5]:
# Dividir os dados em conjuntos de treinamento e teste
def train_test_split(X, y, test_size=0.2, random_state=None):
    if random_state is not None:
        np.random.seed(random_state)
    n_samples = X.shape[0]
    n_test = int(test_size * n_samples)
    test_indices = np.random.choice(n_samples, n_test, replace=False)
    train_indices = np.delete(np.arange(n_samples), test_indices)
    X_train, X_test = X[train_indices], X[test_indices]
    y_train, y_test = y[train_indices], y[test_indices]
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinar o modelo de árvore de decisão
decision_tree = DecisionTree(max_depth=6)
decision_tree.fit(X_train, y_train)

# Testar o modelo
predictions = decision_tree.predict(X_test)

# Avaliar o desempenho do modelo (por exemplo, acurácia)
accuracy = np.mean(predictions == y_test)
print("Acurácia do modelo:", accuracy) 


Acurácia do modelo: 0.45454545454545453


>Este código divide os dados em conjuntos de treinamento e teste, treina um modelo de árvore de decisão com base nos dados de treinamento e testa o modelo com os dados de teste. Finalmente, ele calcula e imprime a acurácia do modelo. Certifique-se de substituir 'penguins.csv' pelo caminho do seu arquivo de dados.

`Prompt 3: Preciso dos resultados de matriz de confusão, acurácia, recall, precisão e f1-score deste algoritmo`

>Para calcular a matriz de confusão, acurácia, recall, precisão e F1-score, você pode usar funções disponíveis em bibliotecas como Scikit-learn ou implementá-las manualmente. Aqui está um exemplo de como você pode calcular essas métricas manualmente:

In [6]:
# Calcular a matriz de confusão
def confusion_matrix(y_true, y_pred):
    num_classes = len(np.unique(y_true))
    conf_matrix = np.zeros((num_classes, num_classes), dtype=int)
    for true, pred in zip(y_true, y_pred):
        conf_matrix[true, pred] += 1
    return conf_matrix

# Calcular a acurácia
def accuracy_score(y_true, y_pred):
    conf_matrix = confusion_matrix(y_true, y_pred)
    correct_predictions = np.diag(conf_matrix).sum()
    total_predictions = conf_matrix.sum()
    accuracy = correct_predictions / total_predictions
    return accuracy

# Calcular o recall
def recall_score(y_true, y_pred, average='macro'):
    conf_matrix = confusion_matrix(y_true, y_pred)
    recall_per_class = np.diag(conf_matrix) / np.sum(conf_matrix, axis=1)
    if average == 'macro':
        recall = np.mean(recall_per_class)
    elif average == 'micro':
        recall = np.sum(np.diag(conf_matrix)) / np.sum(conf_matrix)
    return recall

# Calcular a precisão
def precision_score(y_true, y_pred, average='macro'):
    conf_matrix = confusion_matrix(y_true, y_pred)
    precision_per_class = np.diag(conf_matrix) / np.sum(conf_matrix, axis=0)
    if average == 'macro':
        precision = np.mean(precision_per_class)
    elif average == 'micro':
        precision = np.sum(np.diag(conf_matrix)) / np.sum(conf_matrix)
    return precision

# Calcular o F1-score
def f1_score(y_true, y_pred, average='macro'):
    precision = precision_score(y_true, y_pred, average)
    recall = recall_score(y_true, y_pred, average)
    f1 = 2 * (precision * recall) / (precision + recall)
    return f1

# Fazer previsões usando o modelo de árvore de decisão
predictions = decision_tree.predict(X_test)

# Calcular as métricas
conf_matrix = confusion_matrix(y_test, predictions)
accuracy = accuracy_score(y_test, predictions)
recall = recall_score(y_test, predictions)
precision = precision_score(y_test, predictions)
f1 = f1_score(y_test, predictions)

print("Matriz de Confusão:")
print(conf_matrix)
print("Acurácia:", accuracy)
print("Recall:", recall)
print("Precisão:", precision)
print("F1-score:", f1)


Matriz de Confusão:
[[30  0  0]
 [13  0  0]
 [23  0  0]]
Acurácia: 0.45454545454545453
Recall: 0.3333333333333333
Precisão: nan
F1-score: nan


C:\Users\eduar\AppData\Local\Temp\ipykernel_19024\2494088576.py:30: RuntimeWarning: invalid value encountered in divide
  precision_per_class = np.diag(conf_matrix) / np.sum(conf_matrix, axis=0)


***
## _Aplicando o Código_

***
## _Análise e Considerações_